# Week 5: Branching Processes

**Objective:** Understand how to model population growth and extinction using branching processes, calculate the expected population size, and determine the probability of extinction.

## Step 1: Build Intuition

Imagine an ancient family surname. In each generation, every male member has a random number of sons who will carry on the name. Some may have no sons, some may have one, some may have several. The family name survives only if the line of male descendants continues.

A **branching process** is a mathematical model for this exact scenario. It tracks the size of a population that evolves in discrete generations. Each individual in one generation produces a random number of "offspring" for the next generation, and then "dies" (is no longer counted).

**Key Questions:**
- Will the family name eventually die out (go extinct)?
- What is the expected number of descendants in a future generation?

This model applies to many things: spread of a virus, replication of DNA, or even a nuclear chain reaction.

## Step 2: Understand the Core Idea

A branching process is built on a simple, powerful rule:

**The population size in generation `n+1` is the sum of the offspring of all individuals in generation `n`.**

To define the process, we need two things:
1.  **Initial Population Size (Z₀):** The number of individuals we start with (usually `Z₀ = 1`).
2.  **Offspring Distribution:** A probability distribution that describes how many offspring a single individual produces. This distribution is the same for all individuals in all generations, and the number of offspring for each individual is an independent random variable.

## Step 3: Learn the Definitions and Formulas

**Definition: Branching Process**
A branching process is a discrete-time Markov chain \({Z_n, n \ge 0}\) representing the population size in generation \(n\). It is defined by:
- \(Z_0 = 1\) (or some other starting integer).
- \(Z_{n+1} = \sum_{i=1}^{Z_n} X_{i,n}\), where \(X_{i,n}\) is the number of offspring of the \(i\)-th individual in generation \(n\).
- All \(X_{i,n}\) are independent, identically distributed random variables drawn from the **offspring distribution**.

--- 

**Expected Population Size**
Let \(\mu\) be the mean of the offspring distribution (the average number of offspring per individual). The expected population size in generation \(n\) is:
$$ E[Z_n] = \mu^n Z_0 $$
If we start with one individual (\(Z_0=1\)), then \(E[Z_n] = \mu^n\).

--- 

**Probability of Extinction**
The long-term behavior of the process depends critically on the mean offspring number, \(\mu\).

1.  If **\(\mu < 1\)**: The population is expected to shrink. Extinction is **certain** (probability of extinction is 1).
2.  If **\(\mu = 1\)**: The population is expected to stay constant, but random fluctuations will eventually drive it to 0. Extinction is **certain** (unless each individual produces exactly 1 offspring with probability 1).
3.  If **\(\mu > 1\)**: The population is expected to grow. There is a **chance of survival**. The probability of extinction, \(q\), is the smallest non-negative root of the equation \(g(s) = s\), where \(g(s)\) is the probability generating function of the offspring distribution.

## Step 4: Apply and Practice

Let's simulate a branching process to see this behavior in action. We'll assume the number of offspring for any individual follows a **Poisson distribution**, which is a common choice for modeling births or arrivals.

The mean of a Poisson distribution is its parameter, \(\lambda\). So, \(\mu = \lambda\).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

### Part A: Simulating the Process

Let's write a function to simulate one path of a branching process.

In [ ]:
def simulate_branching_process(start_pop, mu, num_generations):
    """
    Simulates a branching process with a Poisson offspring distribution.
    
    Args:
        start_pop (int): The initial population size, Z_0.
        mu (float): The mean of the Poisson offspring distribution (lambda).
        num_generations (int): The number of generations to simulate.
        
    Returns:
        list: A list of population sizes for each generation.
    """
    population_path = [start_pop]
    current_pop = start_pop
    
    for _ in range(num_generations):
        if current_pop == 0:
            # Once extinct, stays extinct
            population_path.append(0)
            continue
            
        # Each individual in the current population has a random number of offspring
        # drawn from a Poisson distribution with mean mu.
        offspring = np.random.poisson(mu, size=current_pop)
        
        # The next generation's population is the sum of all offspring
        current_pop = np.sum(offspring)
        population_path.append(current_pop)
        
    return population_path

### Part B: Visualizing Extinction and Explosion

Let's simulate the three cases for \(\mu\) to see the different behaviors.

1.  **Subcritical (\(\mu < 1\)):** Population dies out.
2.  **Critical (\(\mu = 1\)):** Population dies out (usually).
3.  **Supercritical (\(\mu > 1\)):** Population either dies out or explodes.

In [ ]:
def plot_simulations(mu, num_sims, num_gens, ax):
    """Helper function to run and plot multiple simulations."""
    extinction_count = 0
    for _ in range(num_sims):
        path = simulate_branching_process(start_pop=1, mu=mu, num_generations=num_gens)
        if path[-1] == 0:
            extinction_count += 1
        ax.plot(path, alpha=0.5)
    
    # Plot theoretical expectation
    generations = np.arange(num_gens + 1)
    expected_path = [mu**n for n in generations]
    ax.plot(generations, expected_path, color='black', linestyle='--', label=f'E[Z_n] = {mu}^n')
    
    ax.set_title(f'Supercritical Case ($\mu = {mu}$)')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Population Size')
    ax.set_ylim(0, 100) # Set a y-limit to see the extinctions clearly
    ax.legend()
    ax.grid(True)
    print(f"For mu={mu}, extinction occurred in {extinction_count}/{num_sims} simulations.")

# --- Simulation Parameters ---
N_SIMS = 50
N_GENS = 20

fig, axes = plt.subplots(1, 3, figsize=(20, 6), sharey=True)

# Case 1: Subcritical
mu_sub = 0.8
plot_simulations(mu_sub, N_SIMS, N_GENS, axes[0])
axes[0].set_title(f'Subcritical Case ($\mu = {mu_sub}$)')

# Case 2: Critical
mu_crit = 1.0
plot_simulations(mu_crit, N_SIMS, N_GENS, axes[1])
axes[1].set_title(f'Critical Case ($\mu = {mu_crit}$)')

# Case 3: Supercritical
mu_super = 1.2
plot_simulations(mu_super, N_SIMS, N_GENS, axes[2])
axes[2].set_title(f'Supercritical Case ($\mu = {mu_super}$)')

plt.suptitle('Branching Process Simulations', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

**Interpretation of the Plots:**

- **Subcritical (μ=0.8):** As expected, all simulation paths quickly go to zero. The expected population (black dashed line) also trends towards zero.
- **Critical (μ=1.0):** The expected population is constant at 1. However, nearly every single simulation path eventually hits zero. This is a key insight: a fair game (on average) still leads to extinction due to random variation.
- **Supercritical (μ=1.2):** This is the most interesting case. The expected population grows exponentially. But the individual paths show an "all-or-nothing" behavior. Many paths still go extinct early on. However, the few that survive tend to grow very large, very quickly. The average is pulled up by these explosive survivors.

## Summary & Next Steps

In this notebook, we've learned:
1.  A **Branching Process** models population dynamics where individuals in one generation produce a random number of offspring for the next.
2.  The long-term behavior is governed by \(\mu\), the mean of the offspring distribution.
3.  If \(\mu \le 1\), extinction is almost certain.
4.  If \(\mu > 1\), there is a non-zero probability of survival and explosive growth, but also a significant chance of early extinction.
5.  The expected population size \(E[Z_n] = \mu^n\) can be misleading, as it averages the many extinctions with the few explosions.

In **Week 6**, we will shift our focus to continuous-time processes and introduce the **Poisson Process**, a fundamental model for counting random events occurring over time.